In [ ]:
pip install easyocr

In [ ]:
pip install streamlit

In [ ]:
pip install streamlit_option_menu

In [ ]:
pip install pyngrok

In [5]:
import sqlite3
ltconn = sqlite3.connect("/content/ocr.db")
ltcursor = ltconn.cursor()
ltcursor.execute("CREATE TABLE card_data (id INTEGER PRIMARY KEY AUTOINCREMENT, company_name TEXT, card_holder TEXT, designation TEXT, mobile_number VARCHAR(50), email TEXT, website TEXT, area TEXT, city TEXT, state TEXT, pin_code VARCHAR(10), image LONGBLOB)")

In [23]:
%%writefile app.py

import easyocr
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu
import streamlit as st
import pandas as pd
import plotly.express as px
import streamlit as st
from PIL import Image
import matplotlib.pyplot as plt
import sqlite3
import cv2
import re
import base64
import io

def image_preview(image,res):
    for (bbox, text, prob) in res:
      # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))
        cv2.rectangle(image, tl, br, (0, 255, 0), 2)
        cv2.putText(image, text, (tl[0], tl[1] - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    plt.rcParams['figure.figsize'] = (15,15)
    plt.axis('off')
    plt.imshow(image)

# CONVERTING IMAGE TO BINARY TO UPLOAD TO SQL DATABASE
def img_to_encode(filename):
    file = open(filename,'rb').read()
    file = base64.b64encode(file)
    return file

def get_data(res):
    for ind,i in enumerate(res):
        # To get WEBSITE_URL
        if "www " in i.lower():
          data["website"]=res[4] +"." + res[5]
        elif "www." in i.lower():
            data["website"]=res[4]  + res[5]
        elif "WWW" in i:
            data["website"] = "www." + res[5]

        # To get EMAIL ID
        elif "@" in i:
            data["email"]=i

        # To get MOBILE NUMBER
        elif "-" in i:
            data["mobile_number"]=i
            if len(data["mobile_number"]) ==2:
                data["mobile_number"] = " & ".join(data["mobile_number"])

        # To get COMPANY NAME
        elif ind == len(res)-1:
            data["company_name"]=i

        # To get CARD HOLDER NAME
        elif ind == 0:
            data["card_holder"]=i

        # To get DESIGNATION
        elif ind == 1:
            data["designation"]=i

        # To get AREA
        if re.findall('^[0-9].+, [a-zA-Z]+',i):
            data["area"]=i.split(',')[0]
        elif re.findall('[0-9] [a-zA-Z]+',i):
            data["area"]=i
        else:
             data["area"]= res[6] #"No"

        # To get CITY NAME
        match1 = re.findall('.+St , ([a-zA-Z]+).+', i)
        match2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
        match3 = re.findall('^[E].*',i)
        if match1:
            data["city"]=match1[0]
        elif match2:
            data["city"]=match2[0]
        elif match3:
            data["city"]=match3[0]
        else:
            data["city"]= res[7] #"No"

        # To get STATE
        state_match = re.findall('[a-zA-Z]{9} +[0-9]',i)
        if state_match:
              data["state"]=i[:9]
        elif re.findall('^[0-9].+, ([a-zA-Z]+);',i):
            data["state"]=i.split()[-1]
        if len(data["state"])== 2:
            data["state"].pop(0)

        # To get PINCODE
        if len(i)>=6 and i.isdigit():
            data["pin_code"]=i
        elif re.findall('[a-zA-Z]{9} +[0-9]',i):
            data["pin_code"]=i[10:]
    data["image"]=img_to_encode(uploaded_card.name)
    return data

icon = Image.open("card.jpg")
st.set_page_config(page_title= "BizCardX - Kavitha",
                page_icon= icon,
                layout= "wide",
                initial_sidebar_state= "expanded",
                )
st.subheader(":blue[BizCardX : Extracting Business Card Data with OCR]")
selected = option_menu(None, ["Upload","Modify","Delete","About"],
                       default_index=0,
                       orientation="horizontal",
                       styles={"nav-link": {"font-size": "20px", "text-align": "centre", "margin": "0px", "hover-color": "#36FF33"},
                               "icon": {"font-size": "20px"},
                               "container" : {"max-width": "6000px"},
                                                              "nav-link-selected": {"background-color": "#333CFF"}})


if selected == "Upload":
    st.caption(":blue[Upload a Business Card]")
    uploaded_card = st.file_uploader("Select file (png,jpg,jpg) to upload",label_visibility="collapsed",type=["png","jpeg","jpg"])

    if uploaded_card is not None:
        # INITIALIZING THE EasyOCR READER
        reader = easyocr.Reader(['en'])
        col1,col2 = st.columns(2,gap="large")
        with col1:
            st.caption("Uploaded image")
            st.image(uploaded_card)
        with col2:
            with st.spinner("Image is processing"):
                st.set_option('deprecation.showPyplotGlobalUse', False)
                saved_img = "/content/" + uploaded_card.name
                image = cv2.imread(saved_img)
                res = reader.readtext(saved_img)
                st.success(":blue[Processed image]")
                st.pyplot(image_preview(image,res))

                result = reader.readtext(uploaded_card.name,detail = 0,paragraph=False)

        data = {"company_name" : [],
                "card_holder" : [],
                "designation" : [],
                "mobile_number" :[],
                "email" : [],
                "website" : [],
                "area" : [],
                "city" : [],
                "state" : [],
                "pin_code" : [],
                "image" : []
              }

        data1=get_data(result)

        df = pd.DataFrame.from_dict(data1, orient='index')
        df = df.transpose()

        st.dataframe(df, hide_index=True)

        if st.button("Save to Database"):
            mydb = sqlite3.connect("/content/ocr.db")
            mycursor = mydb.cursor()
            try:
              for i,row in df.iterrows():
                #row["state"]=row["state"][0]
                sql = "INSERT INTO card_data (company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image) VALUES (?,?,?,?,?,?,?,?,?,?,?)"
                mycursor.execute(sql, tuple(row))
                mydb.commit()
            except Exception as e:
              print(e)
            st.success(":blue[Uploaded to database successfully!]")

if selected == "Modify":
    try:
        mydb = sqlite3.connect("/content/ocr.db")
        mycursor = mydb.cursor()
        mycursor.execute("select card_holder from card_data")
        myrows = mycursor.fetchall()

        business_cards = {}
        for row in myrows:
            business_cards[row[0]] = row[0]
        selected_card = st.selectbox(":blue[Select a card holder name to update]", list(business_cards.keys()))
        st.caption(":blue[Update or modify any data below]")
        mycursor.execute("select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image from card_data WHERE card_holder=?",
                        (selected_card,))
        result = mycursor.fetchone()

        # DISPLAYING ALL THE INFORMATIONS
        company_name = st.text_input("Company_Name", result[0])
        card_holder = st.text_input("Card_Holder", result[1])
        designation = st.text_input("Designation", result[2])
        mobile_number = st.text_input("Mobile_Number", result[3])
        email = st.text_input("Email", result[4])
        website = st.text_input("Website", result[5])
        area = st.text_input("Area", result[6])
        city = st.text_input("City", result[7])
        state = st.text_input("State", result[8])
        pin_code = st.text_input("Pin_Code", result[9])

        # Display image
        image = result[10]
        binary_data = base64.b64decode(image)
        image = Image.open(io.BytesIO(binary_data))
        st.image(image)

        if st.button(":blue[Update changes to DB]"):
            # Update the information for the selected business card in the database
            mycursor.execute("""UPDATE card_data SET company_name=?,card_holder=?,designation=?,mobile_number=?,email=?,website=?,area=?,city=?,state=?,pin_code=?
                                WHERE card_holder=?""", (company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,selected_card))
            mydb.commit()
            st.success(":blue[Information updated in database successfully.]")
    except:
        st.warning("There is no data available in the database")

    if st.button(":blue[View updated data]"):
        mycursor.execute("select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
        myrows = mycursor.fetchall()
        updated_df = pd.DataFrame(myrows,columns=["Company_Name","Card_Holder","Designation","Mobile_Number","Email","Website","Area","City","State","Pin_Code"])
        st.dataframe(updated_df, hide_index=True)
if selected == "Delete":
      mydb = sqlite3.connect("/content/ocr.db")
      mycursor = mydb.cursor()
      mycursor.execute("SELECT card_holder FROM card_data")
      result = mycursor.fetchall()
      business_cards = {}
      for row in result:
          business_cards[row[0]] = row[0]
      selected_card = st.selectbox("Select a card holder name to Delete", list(business_cards.keys()))
      st.write(f":blue[You have selected] :red[**{selected_card}'s**]:blue[ card to delete]")
      st.write(":blue[Proceed to delete this card?]")

      if st.button(":blue[Yes Delete Business Card]"):
          mycursor.execute(f"DELETE FROM card_data WHERE card_holder='{selected_card}'")
          mydb.commit()
          st.success(":blue[Business card information deleted from database.]")

if selected == "About":
    st.caption(":blue[Overview:]")
    st.caption(":blue[This application allows users to upload a business card image and extract relevant information from it using easyOCR.]")
    st.caption(":blue[The extracted informations are company name, card holder name, designation, mobile number, email address, website URL, area, city, state and pin code.]")
    st.caption(":blue[You can view the extracted information in the card.]")
    st.caption(":blue[Technologies :]")
    st.caption(":blue[streamlit GUI, SQL lite,OCR Data Extraction and Python]")


Overwriting app.py


In [7]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.009s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [24]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.148.118.197
npx: installed 22 in 2.856s
your url is: https://big-pets-sleep.loca.lt
